In [3]:
from pylab import *
import matplotlib.pyplot as plt
import os
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb
import pandas as pd
import numpy as np
import pyAgrum as gum


#Whether or not a goal is scored given a chance TO SCORE

bn = gum.BayesNet('Expected Goal')

# chance nodes (random variables)
assist_type = bn.add(gum.LabelizedVariable("assist_type", "assist_type", 3)) # Pass,Cross,Through ball
situation = bn.add(gum.LabelizedVariable("situation", "situation", 3)) # Open Play, Counter Attack, Set Piece
body_part = bn.add(gum.LabelizedVariable("body_part", "body_part", 2)) #Feet ,  Head
location = bn.add(gum.LabelizedVariable("location", "location", 4)) #Central zone:(centre of the box and penalty spot),Wide zone(left-wing, right-wing,left side of the six-yard box, right side of the box, and right side of the six-yard box),Difficult angle zone(difficult angle and long-range, difficult angle on the left, and difficult angle on the right),Long-range zone(long-range, more than 35 yards, and more than 40 yards)

shot_on_target = bn.add(gum.LabelizedVariable("shot_on_target", "shot_on_target", 2)) 
is_goal = bn.add(gum.LabelizedVariable("is_goal", "is_goal", 2)) 
goal_keeper_position = bn.add(gum.LabelizedVariable("goal_keeper_position", "goal_keeper_position", 3)) # Good positioning, avergage poisitioning, Bad positioning(Far Off)
offsideflag = bn.add(gum.LabelizedVariable("offsideflag", "offsideflag", 2)) #FlagUp (No Goal) ,  Flag Down(Goal)

bn.addArc(situation, location)
bn.addArc(assist_type, body_part)
bn.addArc(body_part, location)
bn.addArc(location, shot_on_target)
bn.addArc(body_part, shot_on_target)
bn.addArc(assist_type, shot_on_target)
bn.addArc(situation, shot_on_target)

bn.addArc(shot_on_target,is_goal)
bn.addArc(goal_keeper_position,is_goal)
bn.addArc(offsideflag, is_goal)

bn.cpt(offsideflag).fillWith([0.05,0.95]) # flag up, flag down

bn.cpt(goal_keeper_position).fillWith([0.6, 0.3, 0.2])  # good positioning, Average positioning , Bad(Far off )positioning
bn.cpt(assist_type).fillWith([0.5, 0.2, 0.3]) # Pass, Cross, Throughl Ball


bn.cpt(situation).fillWith([0.8, 0.1, 0.1]) # Open Play, Counter Attack, Set Piece



bn.cpt(body_part)[{'assist_type': 0}] = [0.8, 0.2]
bn.cpt(body_part)[{'assist_type': 1}] = [0.3, 0.7]
bn.cpt(body_part)[{'assist_type': 2}] = [0.95, 0.05]
#Situation: Open Play, Body_part: Feet
bn.cpt(location)[{'situation': 0, 'body_part': 0}] = [0.4, 0.4, 0.15, 0.05] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Open Play, Body_part: Head
bn.cpt(location)[{'situation': 0, 'body_part': 1}] = [0.7, 0.25, 0.04, 0.01] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Counter Attack, Body_part: Feet
bn.cpt(location)[{'situation': 1, 'body_part': 0}] = [0.3, 0.5, 0.15, 0.05] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Counter Attack, Body_part: Head
bn.cpt(location)[{'situation': 1, 'body_part': 1}] = [0.6, 0.3, 0.08, 0.02] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Set Piece, Body_part: Feet
bn.cpt(location)[{'situation': 2, 'body_part': 0}] = [0.35, 0.35, 0.2, 0.1] # Central zone, Wide zone, Difficult angle zone, Long-range zone

#Situation: Set Piece, Body_part: Head
bn.cpt(location)[{'situation': 2, 'body_part': 1}] = [0.8, 0.15, 0.04, 0.01] # Central zone, Wide zone, Difficult angle zone, Long-range zone

# Situation: Open Play, Assist_type: Pass
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 0}] = [0.1, 0.9]  # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 1}] = [0.2, 0.8]  # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 2}] = [0.7, 0.3]  # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 3}] = [0.95, 0.05]  # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 1, 'location': 0}] = [0.15, 0.85]  # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 1, 'location': 1}] = [0.3, 0.7]  # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 0, 'body_part': 1, 'location': 3}] = [0.99, 0.01]  # Head, Long-range zone

# Situation: Open Play, Assist_type: Cross
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 0, 'location': 0}] = [0.6, 0.4]  # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 0, 'location': 1}] = [0.4, 0.6]  # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 0, 'location': 2}] = [0.8, 0.2]  # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 0, 'location': 3}] = [0.98, 0.02]  # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 1, 'location': 0}] = [0.05, 0.95]  # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 1, 'location': 1}] = [0.15, 0.85]  # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 1, 'location': 2}] = [0.95, 0.05]  # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 1, 'body_part': 1, 'location': 3}] = [0.99, 0.01]  # Head, Long-range zone

# Situation: Open Play, Assist_type: Through ball
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 0, 'location': 0}] = [0.1, 0.9]  # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 0, 'location': 1}] = [0.3, 0.7]  # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 0, 'location': 2}] = [0.85, 0.15]  # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 0, 'location': 3}] = [0.97, 0.03]  # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 1, 'location': 1}] = [0.4, 0.6]  # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 1, 'location': 2}] = [0.95, 0.05]  # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 0, 'assist_type': 2, 'body_part': 1, 'location': 3}] = [0.99, 0.01]  # Head, Long-range zone

# Situation: Counter Attack, Assist_type: Pass
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 0, 'location': 0}] = [0.08, 0.92]  # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 0, 'location': 1}] = [0.25, 0.75]  # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 0, 'location': 2}] = [0.8, 0.2]  # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 0, 'location': 3}] = [0.95, 0.05]  # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 1, 'location': 1}] = [0.4, 0.6]  # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 0, 'body_part': 1, 'location': 3}] = [0.98, 0.02]  # Head, Long-range zone

#Situation: Counter Attack, Assist_type: Cross
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 0, 'location': 0}] = [0.15, 0.85] # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 0, 'location': 1}] = [0.35, 0.65] # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 0, 'location': 2}] = [0.75, 0.25] # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 0, 'location': 3}] = [0.96, 0.04] # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 1, 'location': 0}] = [0.05, 0.95] # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 1, 'location': 1}] = [0.12, 0.88] # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 1, 'location': 2}] = [0.94, 0.06] # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 1, 'body_part': 1, 'location': 3}] = [0.99, 0.01] # Head, Long-range zone


#Situation: Counter Attack, Assist_type: Through ball
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 0, 'location': 0}] = [0.05, 0.95] # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 0, 'location': 1}] = [0.2, 0.8] # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 0, 'location': 2}] = [0.8, 0.2] # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 0, 'location': 3}] = [0.97, 0.03] # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 1, 'location': 0}] = [0.3, 0.7] # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 1, 'location': 1}] = [0.5, 0.5] # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 1, 'location': 2}] = [0.95, 0.05] # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 1, 'assist_type': 2, 'body_part': 1, 'location': 3}] = [0.99, 0.01] # Head, Long-range zone

#Situation: Set Piece, Assist_type: Pass
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 0, 'location': 0}] = [0.15, 0.85] # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 0, 'location': 1}] = [0.3, 0.7] # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 0, 'location': 2}] = [0.9, 0.1] # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 0, 'location': 3}] = [0.98, 0.02] # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 1, 'location': 0}] = [0.1, 0.9] # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 1, 'location': 1}] = [0.25, 0.75] # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 1, 'location': 2}] = [0.95, 0.05] # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 0, 'body_part': 1, 'location': 3}] = [0.99, 0.01] # Head, Long-range zone

# Situation: Set Piece, Assist_type: Cross
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 0, 'location': 0}] = [0.7, 0.3]  # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 0, 'location': 1}] = [0.5, 0.5]  # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 0, 'location': 2}] = [0.85, 0.15]  # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 0, 'location': 3}] = [0.98, 0.02]  # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 1, 'location': 1}] = [0.6, 0.4]  # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 1, 'body_part': 1, 'location': 3}] = [0.99, 0.01]  # Head, Long-range zone

# Situation: Set Piece, Assist_type: Through ball
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 0}] = [0.15, 0.85]  # Feet, Central zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 1}] = [0.4, 0.6]  # Feet, Wide zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 2}] = [0.8, 0.2]  # Feet, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 0, 'location': 3}] = [0.95, 0.05]  # Feet, Long-range zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 0}] = [0.2, 0.8]  # Head, Central zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 1}] = [0.3, 0.7]  # Head, Wide zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 2}] = [0.9, 0.1]  # Head, Difficult angle zone
bn.cpt(shot_on_target)[{'situation': 2, 'assist_type': 2, 'body_part': 1, 'location': 3}] = [0.98, 0.02]  # Head, Long-range zone

bn.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 0, 'offsideflag': 0}] = [1, 0] # No shot on target, good goalkeeper positioning, offside flag up
bn.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 0, 'offsideflag': 1}] = [1, 0] # No shot on target, good goalkeeper positioning, offside flag down
bn.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 1, 'offsideflag': 0}] = [1, 0] # No shot on target, average goalkeeper positioning, offside flag up
bn.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 1, 'offsideflag': 1}] = [1, 0] # No shot on target, average goalkeeper positioning, offside flag down
bn.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 2, 'offsideflag': 0}] = [1, 0] # No shot on target, bad goalkeeper positioning, offside flag up
bn.cpt(is_goal)[{'shot_on_target': 0, 'goal_keeper_position': 2, 'offsideflag': 1}] = [1, 0] # No shot on target, bad goalkeeper positioning, offside flag down

bn.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 0, 'offsideflag': 0}] = [1, 0] # Shot on target, good goalkeeper positioning, offside flag up
bn.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 0, 'offsideflag': 1}] = [0.6, 0.4] # Shot on target, good goalkeeper positioning, offside flag down
bn.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 1, 'offsideflag': 0}] = [1, 0] # Shot on target, average goalkeeper positioning, offside flag up
bn.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 1, 'offsideflag': 1}] = [0.3, 0.7] # Shot on target, average goalkeeper positioning, offside flag down
bn.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 2, 'offsideflag': 0}] = [1, 0] # Shot on target, bad goalkeeper positioning, offside flag up
bn.cpt(is_goal)[{'shot_on_target': 1, 'goal_keeper_position': 2, 'offsideflag': 1}] = [0.15, 0.85] # Shot on target, bad goalkeeper positioning, offside flag down

bn

ie = gum.VariableElimination(bn)
ie.makeInference()
print (ie.posterior("is_goal"))


# Set the evidence for the query
ie.setEvidence({'situation': 0, 'assist_type': 0, 'body_part': 0, 'location': 0})

# Query the probability of the shot being on target
p_shot_on_target = ie.posterior('shot_on_target')

print(p_shot_on_target)


assist_types = [0, 0, 0]
max_prob = 0
best_assist_type = None

# Set the evidence for the query
ie.setEvidence({'situation':0, 'body_part': 0, 'location': 0})

# Find the assist type with the highest probability of scoring a goal
for at in assist_types:
    ie.setEvidence({'assist_type': at})
    p_goal = ie.posterior('is_goal')
    if p_goal[1] > max_prob:
        max_prob = p_goal[1]
        best_assist_type = at

print("Best Assist Type:", best_assist_type)


best_body_part_prob = -1
best_body_part = None

for part in range(2):  # 0: Feet, 1: Head
    ie.setEvidence({"body_part": part})
    ie.makeInference()
    goal_prob = ie.posterior("is_goal")[1]

    if goal_prob > best_body_part_prob:
        best_body_part_prob = goal_prob
        best_body_part = part

# Print the best body part
if best_body_part == 0:
    print("The best body part for scorring a goal is Feet.")
elif best_body_part == 1:
    print("The best body part for scoring a goal is Head.")

bn


  is_goal          |
0        |1        |
---------|---------|
 0.6230  | 0.3770  |


  shot_on_target   |
0        |1        |
---------|---------|
 0.1000  | 0.9000  |

Best Assist Type: 0
The best body part for scoring a goal is Head.


(pyAgrum.BayesNet<double>@0000025ED35D3FB0) BN{nodes: 8, arcs: 10, domainSize: 1728, dim: 112, mem: 1Ko 648o}